In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
import urllib.request
import re
import json

# 스팀에 있는 국내 게임 리스트

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASAC 빅데이터 6기/STEAM 분석/steam_list.CSV', encoding = 'cp949', header = None)

In [ ]:
# column 이름이 index =1 에 있어서 변경
df = df.drop(index = 0)
df.columns = df.iloc[0]
df = df.drop(index=1)

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
# 데이터와 관련없는 열 삭제
df = df.drop(['-'], axis= 1)

In [ ]:
df.info()

In [ ]:
# null이 너무 많고 게임의 정보가 관련없느 열에 대해서 삭제
df = df.drop(['개발사 / 개발자 2','개발사 / 개발자 3','배급사 2','배급사 3','x','기타'],axis=1)
df

In [ ]:
# 모든 행이 결측치인 경우에 대해서 삭제
df = df.dropna(how='all')
df

In [ ]:
def extract_app_id(url):
    match_1 = re.search(r'/app/(\d+)', url)  # '/app/' 뒤의 숫자를 찾음
    if match_1:
        return match_1.group(1)  # 매칭된 숫자 부분을 반환
    return None  # 숫자를 찾지 못한 경우 None 반환

# 각 URL에서 숫자를 추출하여 새로운 열로 추가
df['app_id'] = df['게임 링크'].apply(extract_app_id)

In [ ]:
# 각 고유번호별 API호출을 위해 저장
korean_id = df.loc[:,'app_id']
korean_id

In [ ]:
# 번호가 없는 경우(dumy)에 대해서 삭제
korean_id.dropna(inplace=True)

In [ ]:
base_url = 'https://steamspy.com/api.php'

for i in korean_id:
  url = base_url + "?request=appdetails&appid=" + str(i)
  temp_res = urllib.request.urlopen(url)
  temp_data = json.loads(temp_res.read())

In [ ]:
def get_app_details(appid):
    """특정 애플리케이션의 세부 정보를 가져옵니다."""
    params = {'request': 'appdetails', 'appid': appid}
    response = requests.get(BASE_URL, params=params)
    return response.json()

In [ ]:
# 각 고유번호 API호출 받아 리스트형태로 저장
tot_data= []

for i in korean_id:
    url = f"{base_url}?request=appdetails&appid={i}"
    temp_res = urllib.request.urlopen(url)
    temp_data = json.loads(temp_res.read())

    keys = list(temp_data.keys()) # 변수가 너무 많기에 list형태로 변수명 저장

    key_data = []
    for key in keys:
      value = temp_data.get(key,"") #
      key_data.append(value)

    tot_data.append(key_data)

In [ ]:
ko_game = pd.DataFrame(data = tot_data, columns = keys)

In [ ]:
ko_game.info()

In [ ]:
ko_game.head()

In [ ]:
ko_game.to_csv("/content/korean_steam_game.csv")

In [ ]:
# 제공하는 번역 확인
ko_game.loc[:,"languages"].value_counts()

In [ ]:
ko_game.loc[:,"median_forever"].value_counts()

In [ ]:
ko_game.loc[:,"owners"].value_counts()

In [ ]:
# 최근 정식 출시한 '오구와 비밀의 숲'
# 다른 인디게임들과 다르게 광고를 많이 함
ko_game[ko_game.loc[:,'appid'] == 1985960]

In [ ]:
len(ko_game[ko_game['genre'].apply(lambda x: 'Indie' in x.split(','))])

In [ ]:
len(ko_game[ko_game['tags'].apply(lambda x: 'Indie' in x)])

In [ ]:
# 인디 tag가 있는것과 없는 것들에 대해서 데이터프레임 분리
ko_in_game = ko_game[ko_game["tags"].apply(lambda x : 'Indie' in x)]
ko_in_game.reset_index(inplace = True)
ko_in_game.head()

In [ ]:
ko_not_game =  ko_game[ko_game["tags"].apply(lambda x : 'Indie' not in x)]
ko_not_game.reset_index(inplace = True)
ko_not_game.head()

In [ ]:
int(ko_in_game.loc[:,'owners'].str.split(" .. ")[0][1].replace(",", ""))

In [ ]:
ko_in_game['owners'].apply(lambda x: (int(x.split(" .. ")[0].replace(",", "")) + int(x.split(" .. ")[1].replace(",", "")))/2)

In [ ]:
ko_in_game.loc[:,'owners'].apply(lambda x : (int(x.split(" .. ")[0]) + int( x.split(" .. ")[1].replace(",",)))/2)

In [ ]:
ko_in_game['owners']